# 加载机械臂库文件
# Load the robot arm library file

In [ ]:
#!/usr/bin/env python3
#coding=utf-8
import time
from Arm_Lib import Arm_Device

# 获取机械臂的对象
# Get the object of the robotic arm
Arm = Arm_Device()
time.sleep(.1)

# 加载语音识别和语音播报库文件
# Load speech recognition and speech broadcast library files

In [ ]:
import smbus
bus = smbus.SMBus(1)

i2c_addr = 0x0f   #Speech recognition module address
asr_add_word_addr  = 0x01   #Entry add address
asr_mode_addr  = 0x02   #Recognition mode setting address, the value is 0-2, 0: cyclic recognition mode 1: password mode, 2: button mode, the default is cyclic detection
asr_rgb_addr = 0x03   #RGB lamp setting address, need to send two bits, the first one is directly the lamp number 1: blue 2: red 3: green
                      #The second byte is brightness 0-255, the larger the value, the higher the brightness
asr_rec_gain_addr  = 0x04    #Identification sensitivity setting address, sensitivity can be set to 0x00-0x7f, the higher the value, the easier it is to detect but the easier it is to misjudge
                             #It is recommended to set the value to 0x40-0x55, the default value is 0x40
asr_clear_addr = 0x05   #Clear the operation address of the power-off cache, clear the cache area information before entering the information
asr_key_flag = 0x06  #Used in key mode, set the startup recognition mode
asr_voice_flag = 0x07   #Used to set whether to turn on the recognition result prompt sound
asr_result = 0x08  #Recognition result storage address
asr_buzzer = 0x09  #Buzzer control register, 1 bit is on, 0 bit is off
asr_num_cleck = 0x0a #Check the number of entries
asr_vession = 0x0b #firmware version number
asr_busy = 0x0c #Busy and busy flag

i2c_speech_addr = 0x30   #语音播报模块地址
speech_date_head = 0xfd

# 定义播报参数控制函数
# Define broadcast parameter control function

In [ ]:
def I2C_WriteBytes(str_):
    global i2c_speech_addr
    for ch in str_:
        try:
            bus.write_byte(i2c_speech_addr,ch)
            time.sleep(0.01)
        except:
            print("write I2C error")



EncodingFormat_Type = {
                        'GB2312':0x00,
                        'GBK':0X01,
                        'BIG5':0x02,
                        'UNICODE':0x03
                        }
def Speech_text(str_,encoding_format):
    str_ = str_.encode('gb2312')   
    size = len(str_)+2
    DataHead = speech_date_head
    Length_HH = size>>8
    Length_LL = size & 0x00ff
    Commond = 0x01
    EncodingFormat = encoding_format

    Date_Pack = [DataHead,Length_HH,Length_LL,Commond,EncodingFormat]

    I2C_WriteBytes(Date_Pack)

    I2C_WriteBytes(str_)

def SetBase(str_):
    str_ = str_.encode('gb2312')   
    size = len(str_)+2

    DataHead = speech_date_head
    Length_HH = size>>8
    Length_LL = size & 0x00ff
    Commond = 0x01
    EncodingFormat = 0x00

    Date_Pack = [DataHead,Length_HH,Length_LL,Commond,EncodingFormat]

    I2C_WriteBytes(Date_Pack)

    I2C_WriteBytes(str_)

def TextCtrl(ch,num):
    if num != -1:
        str_T = '[' + ch + str(num) + ']'
        SetBase(str_T)
    else:
        str_T = '[' + ch + ']'
        SetBase(str_T)


ChipStatus_Type = {
                    'ChipStatus_InitSuccessful':0x4A,#初始化成功回传#Initialization successful return
                    'ChipStatus_CorrectCommand':0x41,#收到正确的命令帧回传 #Receive the correct command frame and return it
                    'ChipStatus_ErrorCommand':0x45,#收到不能识别命令帧回传 Unrecognized command frame received
                    'ChipStatus_Busy':0x4E,#芯片忙碌状态回传 Chip busy status return
                    'ChipStatus_Idle':0x4F #芯片空闲状态回传  Chip idle status return    
                }

def GetChipStatus():
    global i2c_speech_addr
    AskState = [0xfd,0x00,0x01,0x21]
    try:
        I2C_WriteBytes(AskState)
        time.sleep(0.05)
    except:
        print("I2CRead_Write error")


    try:
        Read_result = bus.read_byte(i2c_speech_addr)
        return Read_result
    except:
        print("I2CRead error")

Style_Type = {
                'Style_Single':0,#为 0，一字一顿的风格# is 0, word by word style
                'Style_Continue':1#为 1，正常合成# is 1, normal synthesis
                }#合成风格设置 [f?]#Synthesis style settings [f?]

def SetStyle(num):
    TextCtrl('f',num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)   


Language_Type = {
                'Language_Auto':0,#为 0，自动判断语种# is 0, automatically determine the language
                'Language_Chinese':1,#为 1，阿拉伯数字、度量单位、特殊符号等合成为中文# is 1, Arabic numerals, measurement units, special symbols, etc. are synthesized into Chinese
                'Language_English':2
                }

def SetLanguage(num):
    TextCtrl('g',num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)

Articulation_Type = {
                'Articulation_Auto':0,
                'Articulation_Letter':1,
                'Articulation_Word':2
                }#设置单词的发音方式 [h?] #Set the pronunciation method of the word [h?]

def SetArticulation(num):
    TextCtrl('h',num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)


Spell_Type = {
                'Spell_Disable':0,#为 0，不识别汉语拼音# is 0, Chinese Pinyin is not recognized
                'Spell_Enable':1
                }#设置对汉语拼音的识别 [i?]#Set up recognition of Chinese Pinyin [i?]

def SetSpell(num):
    TextCtrl('i',num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)


Reader_Type = {
                'Reader_XiaoYan':3,
                'Reader_XuJiu':51,
                'Reader_XuDuo':52,
                'Reader_XiaoPing':53,
                'Reader_DonaldDuck':54,
                'Reader_XuXiaoBao':55#              
                }#选择发音人 [m?]#Select the speaker [m?]

def SetReader(num):
    TextCtrl('m',num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)


NumberHandle_Type = {
                'NumberHandle_Auto':0,
                'NumberHandle_Number':1,
                'NumberHandle_Value':2
                }#设置数字处理策略 [n?]#Set number processing strategy [n?]

def SetNumberHandle(num):
    TextCtrl('n',num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)



ZeroPronunciation_Type = {
                'ZeroPronunciation_Zero':0,
                'ZeroPronunciation_O':1
                }

def SetZeroPronunciation(num):
    TextCtrl('o',num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)



NamePronunciation_Type = {
                'NamePronunciation_Auto':0,
                'NamePronunciation_Constraint':1
                }


def SetNamePronunciation(num):
    TextCtrl('r',num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)

#设置语速 [s?] ? 为语速值，取值：0～10
#Set speaking speed [s?] ? is the speaking speed value, value: 0~10
def SetSpeed(speed):
    TextCtrl('s',speed)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)


#设置语调 [t?] ? 为语调值，取值：0～10
#Set intonation [t?] ? is the intonation value, value: 0~10
def SetIntonation(intonation):
    TextCtrl('t',intonation)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)

#设置音量 [v?] ? 为音量值，取值：0～10
#Set volume [v?] ? is the volume value, value: 0~10
def SetVolume(volume):
    TextCtrl('v',volume)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)


OnePronunciation_Type = {
                'OnePronunciation_Yao':0,
                'OnePronunciation_Yi':1
                }

def SetOnePronunciation(num):
    TextCtrl('y',num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)


Rhythm_Type = {
                'Rhythm_Diasble':0,
                'Rhythm_Enable':1
                }

def SetRhythm(num):
    TextCtrl('z',num)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)

#恢复默认的合成参数 [d] 所有设置（除发音人设置、语种设置外）恢复为默认值
#Restore default synthesis parameters [d] Restore all settings (except speaker settings and language settings) to default values
def SetRestoreDefault():
    TextCtrl('d',-1)
    while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
        time.sleep(0.002)

# 定义语音控制函数
# Define voice control function

In [ ]:
#Write entry
def AsrAddWords(idnum,str):
    global i2c_addr
    global asr_add_word_addr
    words = []
    words.append(asr_add_word_addr)
    words.append(len(str) + 2)
    words.append(idnum)
    for  alond_word in str:
        words.append(ord(alond_word))
    words.append(0)
    print(words)
    for date in words:
        bus.write_byte (i2c_addr, date)
        time.sleep(0.03)

#Set RGB
def RGBSet(R,G,B):
    global i2c_addr
    global asr_rgb_addr
    date = []
    date.append(R)
    date.append(G)
    date.append(B)
    print(date)
    bus.write_i2c_block_data (i2c_addr,asr_rgb_addr,date)

#Read result
def I2CReadByte(reg):
    global i2c_addr
    bus.write_byte (i2c_addr, reg)
    time.sleep(0.05)
    Read_result = bus.read_byte (i2c_addr)
    return Read_result

#Wait busy
def Busy_Wait():
    busy = 255
    while busy != 0:
        busy = I2CReadByte(asr_busy)
        print(asr_busy)	

# 清除掉电缓存区中的词条和模块模式数据，这部分第一次使用写入即可，后续如果不需要在更改设置可以把1 设置为0，或者跳过，之后设置模块的灵敏度和识别提示声的开关，之后亮起模块的RGB 灯为白色1s 和鸣笛1s，并播报"初始化完成，请发布指令"。
# Clear the entries and module mode data in the power-off cache area. This part can be written for the first time. If you do not need to change the settings later, you can set 1 to 0 or skip it, and then set the sensitivity and recognition of the module. Turn on the prompt sound, and then the module's RGB light will turn white for 1s and the siren will sound for 1s, and "Initialization completed, please issue instructions" will be broadcast.

In [ ]:
'''
The mode and phrase have the function of power-down save, if there is no modification after the first entry, you can change 1 to 0
'''
cleck = 1

if 1:
    bus.write_byte_data(i2c_addr, asr_clear_addr, 0x40)#Clear the power-down buffer area
    Busy_Wait()#Wait for the module to be free
    print("Cache cleared")
    bus.write_byte_data(i2c_addr, asr_mode_addr, 1)
    Busy_Wait()
    print("The mode is set")
    AsrAddWords(0,"xiao ya")
    Busy_Wait()
    AsrAddWords(1,"liang hong deng")
    Busy_Wait()
    AsrAddWords(2,"liang lv deng")
    Busy_Wait()
    AsrAddWords(3,"liang lan deng")
    Busy_Wait()
    AsrAddWords(4,"bao jing")
    Busy_Wait()
    AsrAddWords(5,"xiang shang")
    Busy_Wait()
    AsrAddWords(6,"xiang xia")
    Busy_Wait()
    AsrAddWords(7,"xiang zuo")
    Busy_Wait()
    AsrAddWords(8,"xiang you")
    Busy_Wait()
    AsrAddWords(9,"jia jin")
    Busy_Wait()
    AsrAddWords(10,"song kai")
    Busy_Wait()
    while cleck != 11:
        cleck = I2CReadByte(asr_num_cleck)
        print(cleck)

bus.write_byte_data(i2c_addr, asr_rec_gain_addr, 0x40)#Set the sensitivity, the recommended value is 0x40-0x55
bus.write_byte_data(i2c_addr, asr_voice_flag, 1)#Set switch sound
bus.write_byte_data(i2c_addr, asr_buzzer, 1)#buzzer
RGBSet(255,255,255)
time.sleep(1)
RGBSet(0,0,0)
bus.write_byte_data(i2c_addr, asr_buzzer, 0)#buzzer

SetReader(Reader_Type["Reader_XiaoPing"])
SetVolume(8)
Speech_text("初始化完成，请发布指令",EncodingFormat_Type["GB2312"])
while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
    time.sleep(0.1)  

# 循环检测声音
# Loop detection sound

In [ ]:
servo_1 = 90
def main():
    while True:
        global servo_1
        result = I2CReadByte(asr_result)
        #print(result)
        if result == 1:
            Arm.Arm_RGB_set(50, 0, 0) 
            Speech_text("好的，已亮红灯",EncodingFormat_Type["GB2312"])
            while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
                time.sleep(0.1)  
        elif result == 2:
            Arm.Arm_RGB_set(0, 50, 0) #RGB亮绿灯#RGB light green
            Speech_text("好的，已亮绿灯",EncodingFormat_Type["GB2312"])
            while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
                time.sleep(0.1)  
        elif result == 3:
            Arm.Arm_RGB_set(0, 0, 50) #RGB亮蓝灯#RGB bright blue light
            Speech_text("好的，已亮蓝灯",EncodingFormat_Type["GB2312"])
            while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
                time.sleep(0.1)  
        elif result == 4:
            Arm.Arm_Buzzer_On(3)
            time.sleep(.5)
            Speech_text("好的，已开启警报",EncodingFormat_Type["GB2312"])
            while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
                time.sleep(0.1) 
        elif result == 5:
            Arm.Arm_serial_servo_write(2, 90, 1000)
            time.sleep(.001)
            Arm.Arm_serial_servo_write(3, 90, 1000)
            time.sleep(.001)
            Arm.Arm_serial_servo_write(4, 90, 1000)
            time.sleep(1)
            Speech_text("好的，已控制向上",EncodingFormat_Type["GB2312"])
            while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
                time.sleep(0.1) 
        elif result == 6:
            Arm.Arm_serial_servo_write(2, 50, 1000)
            time.sleep(.001)
            Arm.Arm_serial_servo_write(3, 50, 1000)
            time.sleep(.001)
            Arm.Arm_serial_servo_write(4, 50, 1000)
            time.sleep(1)
            Speech_text("好的，已控制向下",EncodingFormat_Type["GB2312"])
            while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
                time.sleep(0.1) 
        elif result == 7:
            servo_1 += 30
            if servo_1 >= 120:
                servo_1 = 120
            Arm.Arm_serial_servo_write(1, servo_1, 500)
            time.sleep(1)
            Speech_text("好的，已控制向左",EncodingFormat_Type["GB2312"])
            while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
                time.sleep(0.1) 
        elif result == 8:
            servo_1 -= 60
            if servo_1 <= 60:
                servo_1 = 60
            Arm.Arm_serial_servo_write(1, servo_1, 500)
            time.sleep(1)
            Speech_text("好的，已控制向右",EncodingFormat_Type["GB2312"])
            while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
                time.sleep(0.1) 
        elif result == 9:
            Arm.Arm_serial_servo_write(6, 180, 500)
            time.sleep(1)
            Speech_text("好的，已控制夹紧",EncodingFormat_Type["GB2312"])
            while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
                time.sleep(0.1) 
        elif result == 10:
            Arm.Arm_serial_servo_write(6, 90, 500)
            time.sleep(1)
            Speech_text("好的，已控制松开",EncodingFormat_Type["GB2312"])
            while GetChipStatus() != ChipStatus_Type['ChipStatus_Idle']:
                time.sleep(0.1) 

        time.sleep(0.5)
        #print(" END OF LINE! ")
try :
    main()
except KeyboardInterrupt:
    # 释放Arm对象
    # Release the Arm object
    del Arm
    del bus
    print(" Program closed! ")
    pass
